In [1]:
using JuMP, Gurobi

INFO: Precompiling module JuMP.
INFO: Precompiling module Gurobi.


### Create th OCT in the paper with a toy dataset

In [61]:
# generate XOR-like data
function f(x1::Float64, x2::Float64)
    if (x1 <= 0.5 && x2 <= 0.5) || (x1 > 0.5 && x2 > 0.5)
        return 0
    end
    return 1     
end

k = 2  # number of classes
p = 2  # number of features
n = 10  # number of data points
x = rand(n, p)
y = diag([f(x1, x2) for x1 = x[:, 1], x2 = x[:, 2]])
Y = reshape(vcat(-y * 2 + 1, y * 2 - 1), (n, p));

In [97]:
# set up model
m = Model(solver=GurobiSolver())

# parameters 
D = 2  # depth
N_min = 1  # min number of data points in each leaf
alpha = 1  # complexity penalty weight

T = 2 ^ (D + 1) - 1  # number of nodes in the tree
Tb = convert(Int, floor(T/2))  # number of branch nodes
Tl = T - Tb  # number of leaf nodes

# variables
@variable(m, a[1:p, 1:Tb], Bin)  # vector a for each branch node
@variable(m, b[1:Tb])  # split value b
@variable(m, d[1:Tb], Bin)  # d indicates if a branch node applies split
@variable(m, z[1:n, Tb + 1:T], Bin)  # z indicates if xi is in leaf node t
@variable(m, l[Tb + 1:T], Bin)  # l indicates if leaf node t contains any points
@variable(m, c[1:k, Tb + 1:T], Bin)  # ???

# constraints
@constraint(m, [t = 1:Tb], sum(a[:, t]) == d[t])  # equation (2)
@constraint(m, [t = 1:Tb], b[t] <= d[t])  # equation (3)
@constraint(m, [t = 1:Tb], b[t] >= 0)  # equation (3)
@constraint(m, [t = 2:Tb], d[t] <= d[convert(Int, floor(t/2))])  # equation (5)
@constraint(m, [i = 1:n, t = Tb + 1:T], z[i, t] <= l[t])  # equation (6)
@constraint(m, [t = Tb + 1:T], sum(z[:, t]) >= N_min * l[t])  # equation (7)
@constraint(m, [i = 1:n], sum(z[i, :]) == 1)  # equation (8)

10-element Array{JuMP.ConstraintRef,1}:
 z[1,4] + z[1,5] + z[1,6] + z[1,7] = 1    
 z[2,4] + z[2,5] + z[2,6] + z[2,7] = 1    
 z[3,4] + z[3,5] + z[3,6] + z[3,7] = 1    
 z[4,4] + z[4,5] + z[4,6] + z[4,7] = 1    
 z[5,4] + z[5,5] + z[5,6] + z[5,7] = 1    
 z[6,4] + z[6,5] + z[6,6] + z[6,7] = 1    
 z[7,4] + z[7,5] + z[7,6] + z[7,7] = 1    
 z[8,4] + z[8,5] + z[8,6] + z[8,7] = 1    
 z[9,4] + z[9,5] + z[9,6] + z[9,7] = 1    
 z[10,4] + z[10,5] + z[10,6] + z[10,7] = 1

In [96]:
m

Feasibility problem with:
 * 55 linear constraints
 * 64 variables: 61 binary
Solver is Gurobi